# 5.5 대화형 챗봇 만들기
- 라이브러리: langchain, streamlit, streamlit_chat, faiss-cpu
- 언어모델: gpt-4
- 벡터 데이터베이스: 파이스(FAISS)

>  stream_chat: 챗봇 사용자 인터페이스를 생성하는데 사용  
> faiss-cpu: 벡터 검색을 위한 인덱싱 및 검색 알고리즘
  

## 코드 흐름
> 질문 -> 임베딩 -> 관련문서(파이스 저장) -> 결합(합침) -> (질문 + 관련문서) -> LLM -> 답변생성

In [5]:
import streamlit as st
from streamlit_chat import message
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
import tempfile
from langchain.document_loaders import PyPDFLoader

2024-09-20 13:33:05.017 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [11]:
# 필요시 api key 입력 필요
import os

In [9]:
uploaded_file = st.sidebar.file_uploader("upload", type="pdf")

2024-09-20 14:00:23.854 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 14:00:23.856 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 14:00:23.866 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 14:00:23.891 
  command:

    streamlit run /opt/anaconda3/envs/llm_project/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-09-20 14:00:23.892 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [10]:
# 사용자에 의해 PDF 파일 업로드
if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name  # tmp_file_name -> tmp_file.name 으로 수정
        
    loader = PyPDFLoader(tmp_file_path)
    data = loader.load()
    embeddings = OpenAIEmbeddings()
    vectors = FAISS.from_documents(data, embeddings)

    # ConversationalRetrievalChain으로 대화형 챗봇 구성
    chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(temperature=0, model_name='gpt-4'), retriever=vectors.as_retriever())

    # 문맥 유지를 위해 과거 대화 이력을 추가(append)
    def conversational_chat(query):
        result = chain({
            "question": query,
            "chat_history": st.session_state['history']
        })
        st.session_state['history'].append((query, result["answer"]))
        return result["answer"]

    # 초기 질문과 답변에 대한 처리(PDF 파일이 업로드되면 보이는 화면 처리) 
    if 'history' not in st.session_state:
        st.session_state['history'] = []

    if 'generated' not in st.session_state:
        st.session_state['generated'] = ["안녕하세요! " + uploaded_file.name + "에 관해 질문 주세요."]

    if 'past' not in st.session_state:
        st.session_state['past'] = ["안녕하세요!"]

    # 챗봇 이력에 대한 컨테이너
    response_container = st.container()
    # 사용자가 입력한 문장에 대한 컨테이너
    container = st.container()

    with container:
        with st.form(key='Conv_question', clear_on_submit=True):
            user_input = st.text_input("Query", placeholder="PDF 파일에 대해 얘기해볼까요? (:", key='input')
            submit_button = st.form_submit_button(label='Send')
        # 사용자가 질문을 입력하거나, [Send] 버튼을 눌렀을 때 처리
        if submit_button and user_input:
            output = conversational_chat(user_input)
            # 사용자의 질문이나 LLM에 대한 결과를 계속 추가(append)
            st.session_state['past'].append(user_input)
            st.session_state['generated'].append(output)
    
    # LLM 이 답변을 해야 하는 경우에 대한 처리
    if st.session_state['generated']:
        with response_container:
            for i in range(len(st.session_state['generated'])):
                message(st.session_state["past"][i], is_user=True,
                        key=str(i) + '_user', avatar_style="fun_emoji", seed="Nala")
                message(st.session_state["generated"][i], key=str(i), avatar_style="bottts", seed="Fluffy")